# Fine-tuning de Modelo com LoRA

Este notebook demonstra como realizar fine-tuning de um modelo de linguagem (Qwen2.5-3B-Instruct) usando a técnica LoRA (Low-Rank Adaptation) para uma tarefa específica de extração de campos estruturados a partir de tickets de call center.

## Objetivo

Treinar o modelo para extrair automaticamente campos estruturados (categoria, subcategoria, grupo de atribuição, etc.) a partir de descrições de tickets de TI, retornando os dados em formato JSON.

## Tecnologias Utilizadas

- **Modelo Base**: Qwen/Qwen2.5-3B-Instruct (modelo de linguagem instrucional)
- **Técnica de Fine-tuning**: LoRA (Low-Rank Adaptation) - permite treinar apenas uma pequena parte dos parâmetros
- **Bibliotecas**: 
  - `transformers` - Hugging Face Transformers
  - `peft` - Parameter-Efficient Fine-Tuning
  - `trl` - Transformer Reinforcement Learning (SFTTrainer)
  - `datasets` - Hugging Face Datasets

## Estrutura do Notebook

1. **Configuração e Carregamento**: Configuração inicial e carregamento do dataset
2. **Preparação do Modelo**: Carregamento do tokenizer e modelo base
3. **Configuração LoRA**: Aplicação da técnica LoRA para treinamento eficiente
4. **Preparação dos Dados**: Formatação e tokenização do dataset
5. **Treinamento**: Execução do fine-tuning
6. **Salvamento**: Salvamento do adaptador LoRA treinado

## Formato do Dataset

O dataset deve estar em formato JSONL com o seguinte formato:

```json
{
  "messages": [
    {"role": "user", "content": "Prompt com a descrição do ticket..."},
    {"role": "assistant", "content": "{\"campo1\": \"valor1\", ...}"}
  ]
}
```


## 1. Configuração e Carregamento do Dataset

Primeiro, vamos configurar os parâmetros principais e carregar o dataset de treinamento.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer


# -------------------------------
# CONFIG
# -------------------------------
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
DATA_FILE = "finetune_multioutput_small.jsonl"  # seu arquivo JSONL com formato de mensagens
OUTPUT_DIR = "./qwen2.5-lora"

# Carregar o dataset de treinamento
# O dataset deve estar em formato JSONL com campo "messages"
print("Carregando dataset...")
dataset = load_dataset("json", data_files=DATA_FILE, split="train")
print(f"Dataset carregado: {len(dataset)} exemplos")
print(f"Estrutura do dataset: {dataset.features}")

# Verificar formato do primeiro exemplo para validação
if len(dataset) > 0:
    print("\nPrimeiro exemplo do dataset:")
    print(dataset[0])


## 2. Verificação de Autenticação

Verifica se você está autenticado no Hugging Face Hub (necessário para baixar modelos privados ou fazer upload).


## 3. Carregamento do Tokenizer

O tokenizer é responsável por converter texto em tokens (IDs numéricos) que o modelo pode processar. Configuramos o token de padding caso não exista.


In [ ]:
from huggingface_hub import whoami
print(whoami())


## 4. Carregamento do Modelo Base

Carregamos o modelo Qwen2.5-3B-Instruct. O modelo detecta automaticamente se há GPU disponível e ajusta o tipo de dados (float16 para GPU, float32 para CPU).


In [ ]:
# -------------------------------
# 2. Load tokenizer
# -------------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_auth_token=True)

# Configurar padding token se não existir
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Pad token: {tokenizer.pad_token}")


## 5. Configuração LoRA

**LoRA (Low-Rank Adaptation)** é uma técnica de fine-tuning eficiente que:

- Treina apenas uma pequena fração dos parâmetros do modelo (geralmente < 1%)
- Adiciona matrizes de baixo rank às camadas de atenção
- Reduz drasticamente o uso de memória e tempo de treinamento
- Mantém a qualidade do modelo original

**Parâmetros LoRA**:
- `r=32`: Rank das matrizes LoRA (maior = mais parâmetros treináveis)
- `lora_alpha=32`: Fator de escala para os pesos LoRA
- `target_modules`: Módulos onde LoRA será aplicado (q_proj, v_proj são camadas de atenção)
- `lora_dropout=0.05`: Taxa de dropout para regularização


In [ ]:
# -------------------------------
# 3. Load base model
# -------------------------------
print("Loading base model...")
# Usar float16 apenas se tiver GPU, caso contrário usar float32
model_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Loading model with dtype: {model_dtype}")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto" if torch.cuda.is_available() else None,
    use_auth_token=True, 
    torch_dtype=model_dtype,
    trust_remote_code=True,
)

if torch.cuda.is_available():
    print(f"Model loaded on device: {next(model.parameters()).device}")
else:
    print("Model loaded on CPU")
print(f"Model dtype: {next(model.parameters()).dtype}")



## 6. Configuração dos Argumentos de Treinamento

Configuramos os hiperparâmetros do treinamento:

- **Batch size**: Tamanho do lote (ajustado para 1 com gradient accumulation)
- **Learning rate**: Taxa de aprendizado (2e-4 é um bom valor inicial)
- **Epochs**: Número de épocas de treinamento
- **Gradient accumulation**: Acumula gradientes de múltiplos batches antes de atualizar pesos
- **FP16**: Usa precisão de 16 bits se GPU disponível (economiza memória)


In [ ]:
# -------------------------------
# 4. LoRA configuration
# -------------------------------
# Para Qwen2.5, os módulos de atenção geralmente são: q_proj, k_proj, v_proj, o_proj
# Vamos usar q_proj e v_proj para começar (mais eficiente)
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],  # módulos de atenção do Qwen2.5
    task_type="CAUSAL_LM",
)

print("Aplicando LoRA ao modelo...")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Associar tokenizer ao modelo (necessário para algumas versões do TRL)
if not hasattr(model, 'tokenizer') or model.tokenizer is None:
    model.tokenizer = tokenizer
    print("✓ Tokenizer associado ao modelo")



## 7. Validação do Formato do Dataset

Verificamos se o dataset está no formato correto esperado pelo SFTTrainer (campo "messages" com formato de chat).


## 8. Preparação e Tokenização do Dataset

Esta é uma etapa crítica do processo:

1. **Formatação**: Converte as mensagens do formato chat para texto formatado no padrão do Qwen (`<|im_start|>user`, `<|im_end|>`, etc.)

2. **Tokenização**: Converte o texto em tokens (IDs numéricos) que o modelo pode processar
   - **Importante**: O DataCollator precisa de dados já tokenizados, não texto bruto
   - Usamos `truncation=True` e `max_length=1024` para limitar o tamanho
   - `padding=False` porque o DataCollator fará padding dinâmico durante o treinamento

3. **DataCollator**: Responsável por criar batches com padding adequado durante o treinamento


In [ ]:
# -------------------------------
# 5. Training arguments
# -------------------------------
# Verificar se há GPU disponível
use_fp16 = torch.cuda.is_available()
print(f"Using FP16: {use_fp16} (GPU available: {torch.cuda.is_available()})")

# Usar TrainingArguments do transformers (compatível com SFTTrainer)
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    warmup_steps=100,
    fp16=use_fp16,  # Use fp16 apenas se tiver GPU
    bf16=False,  # Use bf16 se tiver GPU moderna (A100, H100)
    dataloader_num_workers=0,  # Evitar problemas de multiprocessing
    remove_unused_columns=False,  # Manter todas as colunas do dataset
)



## 9. Treinamento

Inicia o processo de fine-tuning. O treinamento pode levar algum tempo dependendo do tamanho do dataset e dos recursos disponíveis.

**Monitoramento**:
- Os logs aparecem a cada `logging_steps` (10 steps)
- Checkpoints são salvos a cada `save_steps` (500 steps)
- O progresso mostra loss, learning rate e outras métricas


## 10. Salvamento do Modelo

Salva o adaptador LoRA treinado e o tokenizer. O adaptador LoRA contém apenas os pesos treinados (muito menor que o modelo completo) e pode ser carregado junto com o modelo base para inferência.

**Estrutura salva**:
- `adapter_config.json`: Configuração do LoRA
- `adapter_model.bin`: Pesos do adaptador LoRA
- `tokenizer files`: Arquivos do tokenizer

**Para usar o modelo treinado**:
```python
from peft import PeftModel
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model = PeftModel.from_pretrained(model, OUTPUT_DIR)
```


In [ ]:
# Verificar se o dataset tem o formato correto de mensagens
# O SFTTrainer espera um campo "messages" com formato de chat
sample = dataset[0]
if "messages" in sample:
    print("✓ Dataset tem formato de mensagens correto")
    print(f"  Exemplo de mensagens: {sample['messages']}")
else:
    print("✗ Dataset não tem formato de mensagens")
    print(f"  Campos disponíveis: {sample.keys()}")
    raise ValueError("O dataset precisa ter o campo 'messages' no formato de chat")


In [ ]:
# -------------------------------
# 6. Trainer (SFT)
# -------------------------------
# Esta versão do TRL usa uma API diferente - precisa usar processing_class para o tokenizer
# e formatar as mensagens em texto antes

from transformers import DataCollatorForLanguageModeling

def format_messages_manual(messages):
    """Formata mensagens manualmente no formato do Qwen"""
    formatted = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            formatted += f"<|im_start|>user\n{content}<|im_end|>\n"
        elif role == "assistant":
            formatted += f"<|im_start|>assistant\n{content}<|im_end|>\n"
    return formatted

# Função para formatar as mensagens do dataset em texto
def format_messages(examples):
    """Converte mensagens do formato chat para texto formatado"""
    texts = []
    for messages in examples["messages"]:
        # Verificar se o tokenizer tem apply_chat_template
        if hasattr(tokenizer, 'apply_chat_template') and tokenizer.chat_template is not None:
            try:
                formatted_text = tokenizer.apply_chat_template(
                    messages, 
                    tokenize=False, 
                    add_generation_prompt=False
                )
            except:
                # Fallback: formatar manualmente
                formatted_text = format_messages_manual(messages)
        else:
            # Formatar manualmente se não tiver chat_template
            formatted_text = format_messages_manual(messages)
        texts.append(formatted_text)
    return {"text": texts}

# Aplicar formatação ao dataset
print("Formatando dataset...")
try:
    dataset_formatted = dataset.map(
        format_messages,
        batched=True,
        remove_columns=dataset.column_names,  # Remove colunas originais
    )
    print(f"✓ Dataset formatado. Exemplo (primeiros 200 chars): {str(dataset_formatted[0])[:200]}...")
except Exception as e:
    print(f"Erro ao formatar dataset: {e}")
    # Tentar sem remover colunas
    dataset_formatted = dataset.map(
        format_messages,
        batched=True,
    )
    print(f"✓ Dataset formatado (mantendo colunas originais)")

# Tokenizar o dataset - CRÍTICO: o DataCollator precisa de dados tokenizados, não texto bruto
print("\nTokenizando dataset...")
def tokenize_function(examples):
    """Tokeniza os textos do dataset"""
    # Tokenizar com padding e truncation
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,  # O DataCollator fará o padding
        max_length=1024,
        return_tensors=None,  # Retornar como listas, não tensors
    )
    return tokenized

dataset_tokenized = dataset_formatted.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],  # Remover texto bruto, manter apenas tokens
    desc="Tokenizando dataset"
)
print(f"✓ Dataset tokenizado. Exemplo de campos: {list(dataset_tokenized[0].keys())}")

# Criar data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Não é masked language modeling
    pad_to_multiple_of=8,  # Otimização para GPUs modernas
)

# Criar o trainer com os parâmetros corretos para esta versão do TRL
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset_tokenized,  # Usar dataset tokenizado, não formatado
    processing_class=tokenizer,  # Tokenizer via processing_class
    data_collator=data_collator,
)

print("✓ SFTTrainer criado com sucesso!")



In [ ]:
# -------------------------------
# 7. Train!
# -------------------------------
print("Starting training...")
print(f"Trainer type: {type(trainer)}")
print(f"Trainer tem tokenizer: {hasattr(trainer, 'tokenizer')}")

try:
    trainer.train()
except Exception as e:
    print(f"Erro durante o treinamento: {e}")
    print(f"Tipo do erro: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    raise




In [ ]:
# -------------------------------
# 8. Save LoRA adapter
# -------------------------------
print("Saving model...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Done!")